In [ ]:
import tensorflow as tf
import dill
import contrastive
from contrastive import utils as contrastive_utils
from acme import specs
import jax
import numpy as np
from scipy.spatial.distance import cdist
from collections import defaultdict
from matplotlib import pyplot as plt

import imageio
from base64 import b64encode
from IPython.display import display, HTML
import tempfile
import tqdm
import matplotlib.pyplot as plt
import jax.numpy as jnp
import gym
from acme.wrappers import gym_wrapper

import metaworld
# from contrastive_rl import classifier_envs
from acme.wrappers import gym_wrapper
from acme.wrappers import step_limit
# from contrastive_rl import classifier_utils
# from metaworld.envs.mujoco.utils import rotation

In [ ]:
# Wasn't able to use the checkpoints that Ben shared since there were some structures
# from Google's internal libaries. Kyle's checkpoints work though!
filename = './checkpoints/learner/'  # <-- Of course, this will be different for you
reader = tf.train.load_checkpoint(filename)
params = reader.get_tensor('learner/.ATTRIBUTES/py_state')
state = dill.loads(params)

In [ ]:
env, obs_dim = contrastive_utils.make_environment('fixed-goal-point_Cross', start_index=0, end_index=-1, seed=25)
spec = specs.make_environment_spec(env)
networks = contrastive.make_networks(spec, obs_dim,
                             actor_min_std=1e-6,  # <-- Make sure this is the same value used for training.
                             use_image_obs=False)

key = jax.random.PRNGKey(0)

# @jax.jit
def _act(obs, key):
  dist_params = networks.policy_network.apply(
    state.policy_params, obs[None])
  key, rng = jax.random.split(key, 2)
  # action = dist_params.mode()  # <-- Use this if you want to act deterministically
  action = networks.sample(dist_params, rng)
  return action[0], key

ts = env.reset()
_act(ts.observation, key)

In [ ]:
state.policy_params.keys()

In [ ]:
# the video writer commands will have to change here. I'd probably recommend using
# imageio. You just make a list of all the images, and then call imageio.mimsave(filename, image_list)

waypoints = defaultdict(list)

for x in range(100):
    # If i change the range here then I get that many new startign configurations
  video_filename = '/tmp/video.mp4'
  writer = []

  vec = []
  ts = env.reset()
  starting_position = env.state
#   print('Starting configuration: {}'.format(env.state))
  goal_position = env.goal
  env._goal_img = ts.observation.copy()
  for t in tqdm.trange(50):
#     print(goal_state, env.state)
    a, key = _act(ts.observation, key)
    ts = env.step(a.to_py())
    # use the following lines for image-based environments.
#     s = ts.observation[:obs_dim]
    g = ts.observation[obs_dim:]
    img = env.render(mode='rgb_array')
#     img = np.concatenate([
#       s.reshape((64, 64, 3)),
#       g.reshape((64, 64, 3))
#     ], axis=1)
    # for state-based environments, use the following
    # img = env.render(mode='rgb_array'), or something like that
    writer.append(img)
    if ts.last():
      print('Ending configuration: {}'.format(env.state))
      ending_position = env.state
      color = 'r'
      print(ending_position, starting_position, cdist([ending_position], [starting_position]))
      if cdist([ending_position], [starting_position])<1.5:
        color = 'g'
        print('close!')
      waypoints['configuration'].append({'starting_position': starting_position, 'goal_position':goal_position, 'color': color})
      break
#   imageio.mimsave('output_{}.gif'.format(x), writer)

#   mp4 = open('output.gif','rb').read()
#   data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
#   html = HTML("""
#   <video width=800 framerate=1 controls>
#         <source src="%s" type="video/mp4">
#   </video>
#   """ % data_url)
#   display(html)
# Not sure if the _dist variable applies to a state based environment? Commenting for now
#   plt.plot(env.environment.environment.environment._dist)
#   plt.show()

In [ ]:
def plot_exploration(waypoints, environment = 'Cross'):
    h, w, walls = env_config(environment)
    x, y, c = [waypoints['configuration'][t]['starting_position'][0] for t in range(len(waypoints['configuration']))], [waypoints['configuration'][t]['starting_position'][1] for t in range(len(waypoints['configuration']))], [waypoints['configuration'][t]['color'] for t in range(len(waypoints['configuration']))]
    g_x, g_y = waypoints['configuration'][0]['goal_position'][0], waypoints['configuration'][0]['goal_position'][1]
    fig = plt.figure(figsize = (10, 10))
    ax = fig.add_subplot()
    ax.scatter(y, x, color = c, marker = '+', s = 50, label = 'start locs')
    ax.scatter(g_y, g_x, color = 'g', marker = 'o', s = 100, label = 'goal loc')
    ax.scatter(walls[1], walls[0], color = 'b', marker = 'x', s = 100, label = 'walls')
    plt.xlabel('Y axis')
    plt.ylabel('X axis')
    plt.title('Goal location: {}, {}'.format(g_y, g_x))
    plt.legend(loc = 'best')
    plt.yticks([idx for idx in range(h)])
    plt.ylim(h - 1, 0)
    plt.xlim(0, w - 1)
    plt.savefig('this.jpg')
    return 0

plot_exploration(waypoints)

In [ ]:
def env_config(environment):
    wall_indices = []
    h, w = WALLS[environment].shape
    wall_x, wall_y = np.where(WALLS[environment]==1)
    return h, w, (wall_x, wall_y)

In [ ]:
WALLS = {
    'Small':  # max_goal_dist = 3
        np.array([[0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0],
                  [0, 0, 0, 0]]),
    'Cross':  # max_goal_dist = 9
        np.array([[0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0]]),
    'FourRooms':  # max_goal_dist = 14
        np.array([[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]),
    'U':  # max_goal_dist = 14
        np.array([[0, 0, 0],
                  [0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0],
                  [1, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0],
                  [0, 1, 0],
                  [0, 0, 0]]),
    'Spiral11x11':  # max_goal_dist = 45
        np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
                  [1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
                  [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0],
                  [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0],
                  [1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
                  [1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0],
                  [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
                  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]),
    'Maze11x11':  # max_goal_dist = 49
        np.array([[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
                  [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
                  [1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0],
                  [1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                  [1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
}